In [1]:
# Import libraries and declare main arrays
import numpy as np
import pandas as pd
from itertools import combinations

metric_dict = {
    "antecedent support": lambda _, sA, __: sA,
    "consequent support": lambda _, sA, __: sC,
    "confidence": lambda sAC, sA, _: sAC/sA,
    "rule support": lambda sAC, _, __: sAC,
    "conf.difference": lambda sAC, sA, sC: abs(metric_dict["confidence"](sAC, sA, sC)-sC),
    "conf.ratio": lambda sAC, sA, sC: 1-np.minimum(sAC/sA, sC)/np.maximum(sAC/sA, sC),
    "lift": lambda sAC, sA, sC: metric_dict["confidence"](sAC, sA, sC)/sC
    }

columns_ordered = ["antecedent support", "consequent support", 
                   "confidence", "rule support", "lift",
                   "conf.difference", "conf.ratio"]

In [2]:
# Apriori function taken from mlxtend library: https://pypi.python.org/pypi/mlxtend
def apriori(df, min_support=0, use_colnames=False):
    X = df.values
    ary_col_idx = np.arange(X.shape[1]-1)+1
    support = (np.sum(X, axis=0) / float(X.shape[0]))
    support=support[support<1]
    support_dict = {1: support[support >= min_support]}
    itemset_dict = {1: ary_col_idx[support >= min_support].reshape(-1, 1)}
    
    max_itemset = 1

    while max_itemset:
        next_max_itemset = max_itemset + 1
        combin = combinations(np.unique(itemset_dict[max_itemset].flatten()),
                              r=next_max_itemset)
        frequent_items = []
        frequent_items_support = []

        for c in combin:
            together = X[:, c].sum(axis=1) == len(c)
            support = together.sum() / float(X.shape[0])
            if support >= min_support:
                frequent_items.append(c)
                frequent_items_support.append(support)

        if frequent_items:
            itemset_dict[next_max_itemset] = np.array(frequent_items)
            support_dict[next_max_itemset] = np.array(frequent_items_support)
            max_itemset = next_max_itemset
        else:
            max_itemset = 0

    all_res = []
    for k in sorted(itemset_dict):
        support = pd.Series(support_dict[k])
        itemsets = pd.Series([i for i in itemset_dict[k]])

        res = pd.concat((support, itemsets), axis=1)
        all_res.append(res)

    res_df = pd.concat(all_res)
    res_df.columns = ['support', 'itemsets']
    if use_colnames:
        mapping = {idx: item for idx, item in enumerate(df.columns)}
        res_df['itemsets'] = res_df['itemsets'].apply(lambda x: [mapping[i]
                                                      for i in x])
    res_df = res_df.reset_index(drop=True)
    return res_df

In [3]:
# Example: load a DSS dataset as a Pandas dataframe (change the dataset name if on a different project)
df = pd.read_csv('Shopping_tabular.txt', delimiter=';')
df.head()

,TICKET,Alcohol,Bakery goods,Fresh Vegetables,Fresh meat,Frozen foods,Milk,Ready made,Snacks,Tinned Goods,Toiletries
0,1,0,0,0,0,0,0,1,1,0,0
1,2,0,0,0,0,0,0,1,0,0,1
2,3,0,0,0,0,0,0,1,1,0,1
3,4,0,1,0,0,0,1,1,0,0,0
4,5,0,0,0,0,0,0,1,0,0,0


In [4]:
# at any time we can show frequent itemsets
apriori(df, min_support=0.2, use_colnames=True)

,support,itemsets
0,0.394402,[Alcohol]
1,0.428753,[Bakery goods]
2,0.402036,[Frozen foods]
3,0.492366,[Ready made]
4,0.474555,[Snacks]
5,0.455471,[Tinned Goods]
6,0.215013,"[Alcohol, Bakery goods]"
7,0.230280,"[Alcohol, Frozen foods]"
8,0.212468,"[Alcohol, Ready made]"
9,0.218830,"[Alcohol, Snacks]"


And it is always advisable to show statistics of the 10 flag variables, to get a first feeling of the data.<BR>
If you show the mean of all of them, you get their relative frequencies (since the are coded 0-1)

In [5]:
df.drop("TICKET", axis=1).mean().sort_values(ascending=False)

Ready made          0.492366
Snacks              0.474555
Tinned Goods        0.455471
Bakery goods        0.428753
Frozen foods        0.402036
Alcohol             0.394402
Milk                0.188295
Toiletries          0.099237
Fresh Vegetables    0.082697
Fresh meat          0.029262
dtype: float64

In [6]:
# Area to change the script parameters
metric="lift"        # metric used for threshold
min_threshold=1      # metric threshold
min_confidence=0.8         # min confidence
min_rulesupport=0       # min rule support (enter 0 to use antecedent support + other metric)
min_support=0.1       # min antecedent support
max_antecedent=5        # define maximum number of antecedent

# calls apriori function to produce, in first hand, the most frequent combinations of products = itemsets
freq=apriori(df, min_rulesupport, use_colnames=True)

keys = freq['itemsets'].values
values = freq['support'].values
frozenset_vect = np.vectorize(lambda x: frozenset(x))
frequent_items_dict = dict(zip(frozenset_vect(keys), values))

In [7]:
# prepare buckets to collect frequent rules
rule_antecedents = []
rule_consequent = []
rule_supports = []

# nsize will store total number of tickets
nsize=len(df.index) 

# iterate over all frequent itemsets
for k in frequent_items_dict.keys():
    sAC = frequent_items_dict[k]
    # iterate to find all possible combinations. len(k) is itemset length
    for idx in range(len(k)-1, 0, -1):
        # extract supports, antecedents and consequent from rules
        for c in combinations(k, r=idx):
            antecedent = frozenset(c)
            consequent = k.difference(antecedent)
            sA = frequent_items_dict[antecedent]
            sC = frequent_items_dict[consequent]
            instances = sA * nsize
            str_antecedent=str(list(c))[2:-2].replace("', u'", ",")
            str_consequent=str(list(consequent))[2:-2].replace("', u'", ",")
            # check thresholds
            if metric_dict[metric](sAC, sA, sC) >= min_threshold:
                if metric_dict['antecedent support'](sAC, sA, sC) >= min_support:
                    if metric_dict['confidence'](sAC, sA, sC) >= min_confidence:
                        if len(antecedent)<=max_antecedent and len(consequent)==1:
                            rule_antecedents.append(str_antecedent.replace("','", ",").replace("', '", ","))
                            rule_consequent.append(str_consequent.replace("','", ",").replace("', '", ","))
                            rule_supports.append([len(antecedent), len(consequent), instances, sAC, sA, sC])


C:\Users\SLO\AppData\Local\Temp\ipykernel_5076\4060651816.py:9: RuntimeWarning: invalid value encountered in scalar divide
  "confidence": lambda sAC, sA, _: sAC/sA,
C:\Users\SLO\AppData\Local\Temp\ipykernel_5076\4060651816.py:13: RuntimeWarning: invalid value encountered in scalar divide
  "lift": lambda sAC, sA, sC: metric_dict["confidence"](sAC, sA, sC)/sC


In [8]:
# if rules were found, generate metrics from rule_supports array
if len(rule_supports)==0:
    print ("EMPTY: No rules were found with the criteria specified.")    
else:
    rule_supports= np.array(rule_supports).T.astype(float)
    numbantec = rule_supports[0]
    numbconseq = rule_supports[1]
    instances = rule_supports[2]
    sAC = rule_supports[3]
    sA = rule_supports[4]
    sC = rule_supports[5]

    # create the final pandas dataframe: dfrules, to store rule list
    dfrules = pd.DataFrame(data=list(zip(rule_antecedents, rule_consequent, instances)), columns=["antecedents", "consequent", "instances"])

    # add other metrics
    for m in columns_ordered:
        dfrules[m] = metric_dict[m](sAC, sA, sC)

    # add number of antecedents and number of consequents
    dfrules2=pd.DataFrame(data=list(zip(numbantec, numbconseq)), columns=["#antec", "#conseq"])
    dfrules=pd.concat([dfrules, dfrules2], axis=1)
    
    # sort by one metric
    dfrules=dfrules.sort_values('confidence', ascending=False)

    # assign row number
    dfrules['ruleid']=''
    for index in dfrules.iterrows():
        dfrules['ruleid']=dfrules.index+1
    
    print (str(len(dfrules)) + " rules found")        

4 rules found


In [9]:
# show rules
dfrules

,antecedents,consequent,instances,antecedent support,consequent support,confidence,rule support,lift,conf.difference,conf.ratio,#antec,#conseq,ruleid
0,"Frozen foods,Milk",Bakery goods,85.0,0.108142,0.428753,0.835294,0.090331,1.948193,0.406541,0.486704,2.0,1.0,1
1,"Tinned Goods,Ready made,Alcohol",Bakery goods,95.0,0.120865,0.428753,0.831579,0.100509,1.939528,0.402826,0.484411,3.0,1.0,2
3,"Tinned Goods,Frozen foods,Snacks",Bakery goods,90.0,0.114504,0.428753,0.822222,0.094148,1.917705,0.393469,0.478543,3.0,1.0,4
2,"Tinned Goods,Alcohol,Bakery goods",Ready made,97.0,0.123410,0.492366,0.814433,0.100509,1.654120,0.322067,0.395449,3.0,1.0,3


In [10]:
# and export them to Excel to use them in Predictions
dfrules=dfrules.sort_values("ruleid")
dfrules.to_excel("Rules.xlsx")